<a href="https://colab.research.google.com/github/GeorgeFane/least-squares-regression/blob/master/Copy_of_Least_Squares_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

jupyter notebook \ --NotebookApp.allow_origin='https://colab.research.google.com' \ --port=8888 \ --NotebookApp.port_retries=0

In [67]:
from jupyter_dash import JupyterDash

import dash
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output
from dash_table import DataTable

from random import uniform

The below function has a similar purpose to group() in my Payoff Matrix Solvers. It takes a string, containing space-separated numbers, and splits them into a list of pairs of floats. In context, it takes a string of coordinates and formats them into (x, y) pairs.

In [68]:
def getPairs(string):
    row=list(map(float, string.split()))
    return [row[i: i+2] for i in range(len(row)) if i%2==0]

Self-explanatory. As one use, it turns my (x, y) pairs into a list of just x-values and a list of just y-values.

In [69]:
def transpose(matrix):
    return [[row[j] for row in matrix] for j in range(len(matrix[0]))]

Let's say you have a few points that you want to fit to a quadratic equation. That means you want to find a, b, and c such that a + bx + cx^2 = y is as accurate a model as possible. Expressed in matrix terms with points (x0, y0), (x1, y1), and (x2, y2):

    [
        [x0^0, x0^1, x0^2],
        [x1^0, x1^1, x1^2],
        [x2^0, x2^1, x2^2],
    ]
    *
    [
        [a],
        [b],
        [c],
    ]
    =
    [
        [y0],
        [y1],
        [y2],
    ]
The below function generates the first matrix given a list of x-values. The first matrix is called A in the above embedded website, so I call the function getA().

In [70]:
def getA(xlist, degrees):
    return [[x**degree for degree in range(degrees+1)] for x in xlist]

This is a simple dot product. It helps with matrix multiplication.

In [71]:
def dot(u, v):
    return sum([x*y for x, y in zip(u, v)])

Because matrix multiplication means a bunch of dot products between the first matrix's rows and the second's columns, I transpose the second so I can more easily perform dot products.

In [72]:
def multiply(a, b):
    return [[dot(row, column) for column in transpose(b)] for row in a]

This combines the corresponding rows of two matrices and returns the augmented matrix.

In [73]:
def augment(a, b):
    return [x+y for x, y in zip(a, b)]

The name for the below function isn't great. It goes through a vector and returns the first nonzero value. 

In [74]:
def getConstant(v):
    for x in v:
        if x!=0:
            return x

I use the below function to help with getting Reduced Row-Echelon Form for matrices. It takes a row vector and divides every entry by some constant, provided by the above function, to make sure it has a leading 1.

This probably should be a void or mutator function, but having it return something makes it easier to test.

In [75]:
def simplify(v):
    constant=getConstant(v)
    return [x/constant for x in v]

The below function performs: row v minus a multiple of row u. I make sure to only pass rows with leading 1s as u, so that the multiple is simply the entry in row v with the same column number as the leading 1 in row u.

In [76]:
def clear(u, v):
    i=u.index(1)
    constant=v[i]
    return [y-constant*x for x, y in zip(u, v)]

Below is my function to change matrices to Reduced Row-Echelon Form. I augment Matrix A with a column vector of corresponding y-values and solve the combined matrix with the below function to get a column vector of coefficients a, b, c...

This function has many possible errors. For one, if the first row starts with a 0, this function doesn't even think of swapping rows. However, this probably won't happen as Matrix A's first column is filled with x^0 terms.

In [77]:
def reduce(matrix):
    height=len(matrix)
    
    for i in range(height):
        matrix[i]=simplify(matrix[i])
        
        for j in range(i+1, height):
            matrix[j]=clear(matrix[i], matrix[j])
            
    for i in range(height):
        for j in range(i+1, height):
            matrix[i]=clear(matrix[j], matrix[i])
            
    return matrix

This wraps several of the above functions into a single function that goes from simple x-values, y-values, and polynomial degree to a complete best-fit polynomial's coefficients.

In [78]:
def getCoefs(x, y, degrees):
    a=getA(x, degrees)
    aT=transpose(a)
    
    b=[[y0] for y0 in y]
    
    aTa=multiply(aT, a)
    aTb=multiply(aT, b)
    
    system=augment(aTa, aTb)
    
    return [row[-1] for row in reduce(system)]

The below function is my poor man's version of NumPy's linspace(). While the latter returns n evenly-spaced numbers between lower and upper bounds, mine generates 99 random numbers between those bounds and then returns the sorted listed. I use this function after I know the equation for the line of best fit. It helps me get the data to plot that line.

In [79]:
def randX(down, up):
    return sorted([uniform(down, up) for i in range(99)])

The below function takes a row vector of x-values, gets the A matrix for them, and returns a row vector of corresponding y-values

CHANGE THIS

In [80]:
f = lambda coefs, x: sum([coef*x**power for power, coef in enumerate(coefs)])

We're finally at the frontend bit of the code!.

The first line initializes the app. By using JupyterDash, I can run my app inline. I actually didn't know I could use that in Google Colab. I simply inferred from the name that I could only use it in Jupyter and didn't realize the truth until importing that notebook into Colab to write all these text cells and copy to Github Gist.

Anyways, app.layout is pretty easy to write because you're just listing all the components that you need. I have lots of breaklines, labels, and input fields, and then a graph and an H1 for a title.

In [81]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app=JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Least-Squares Regression'),
    
    html.Label('Space-Separated Data: '),
    dcc.Input(id='points', value='1 0 2 3 3 7 4 14 5 22'),
    html.Br(),

    html.Label('For example, input points (1, 0), (2, 3), (3, 7), (4, 14), (5, 22) as 1 0 2 3 3 7 4 14 5 22'),
    html.Br(),
    html.Br(),

    html.Div([
        html.Div([
            html.Label('Degrees of Best-Fit Line: '),
            dcc.Input(id='degrees', type='number', value=2),
            
            html.Div(id='eq'),
            
            html.Div(id='coefs', style={'display': 'none'}),
            
            dcc.Graph(id='graph'), 
            html.Br(),   

            html.Label('Model Prediction for x = '),
            dcc.Input(id='x', type='number'),
            html.Br(),
            html.Div(id='y'),
        ], className="six columns"),

        html.Div([
            html.Label('n-th Derivative of Model'),
            dcc.Dropdown(id='derivDrop'),
            html.Br(),
            
            html.Div(
                id='derivs', 
                style={'display': 'none'},
            ),
            
            html.Div(id='derivEq'),
            
            dcc.Graph(id='derivGraph'), 
            html.Br(),   

            html.Label('n-th Derivative of Model at x = '),
            dcc.Input(id='xDeriv', type='number'),
            html.Br(),
            html.Div(id='yDeriv'),
        ], className="six columns"),
    ], className="row"),
])

Below is one of my two frontend functions. This one plots data and shows the best-fit line's equation.

It takes a string of coordinates and the degree of polynomial desired for the best-fit line. It turns that string into a list of (x, y) pairs, then transposes that list to get a list of x-values and a list of y-values. I put those two lists into another list called traces, which I stores data that I will later plot.

I initialize input field for degrees to -1, so that as soon as the user inputs a string of points, those points show up on the graph without a line of best fit. The user can immediately see whether they inputted their points correctly, without any other clutter.

When the user changes the degree field to something nonnegative, my program performs all the linear algebra operations that find the line of best fit. I then get a bunch of x- and y-values along that line and add them to 'traces' for plotting.

I also fill the string 'eq' (meaning equation), so that the user can see their best-fit equation and copy it in case they need it somewhere else. I round each coefficent to 9 places because I've noticed floating-point rounding errors, liked 7.3 showing up as 7.300000000000000001.

In [82]:
getEq = lambda coefs: ' + '.join([f'({round(coef, 9)}) x^{i}' for i, coef in enumerate(coefs)])

In [83]:
import json

@app.callback(
    [
        Output('graph', 'figure'),
        Output('eq', 'children'),
        Output('coefs', 'children'),
    ],
    [
        Input('points', 'value'),
        Input('degrees', 'value'),
    ]
)
def fillChart(points, degrees):
    x, y=transpose(getPairs(points))
    eq=''
    coefs=[0]
          
    traces=[
        dict(
            x=x,
            y=y,
            mode='markers',
            name='Data'
        )
    ]
    
    if degrees>=0:
        coefs=getCoefs(x, y, degrees)

        x1=randX(min(x), max(x))
        y1=[f(coefs, x) for x in x1]

        traces.append(
            dict(
                x=x1,
                y=y1,
                name='Line of Best Fit'
            )
        )
        
        eq=f'Model: f (x) = ' + getEq(coefs)
    
    return (
        dict(
            data=traces,
            layout=dict(
                hovermode='closest'
            )
        ),
        eq,
        json.dumps(coefs),
    )

The below function allows the user to get their model's predictions for any value of x.

In [84]:
@app.callback(
    Output('y', 'children'),
    [
        Input('x', 'value'),
        Input('coefs', 'children'),
    ]
)
def predict(x, coefsString):       
    coefs=json.loads(coefsString)
    return f'f ({x}) = {round(f(coefs, x), 9)}'

In [85]:
derive = lambda coefs: [coef*(power+1) for power, coef in enumerate(coefs[1:])]

In [86]:
@app.callback(
    [
        Output('derivs', 'children'),    
        Output('derivDrop', 'options'), 
    ],
    [
        Input('coefs', 'children'),
    ]
)
def fillDrop(coefsString):       
    coefs=json.loads(coefsString)
    derivs=[coefs]
    while len(derivs[-1])>1:
        derivs.append(derive(derivs[-1]))

    return [
        json.dumps(derivs),
        [dict(label = i, value = i) for i in range(len(derivs))],
    ]    

In [87]:
@app.callback(
    [   
        Output('derivGraph', 'figure'), 
        Output('derivEq', 'children'), 
    ],
    [
        Input('points', 'value'),
        Input('derivs', 'children'),    
        Input('derivDrop', 'value'), 
    ],
)
def plotDeriv(points, derivsString, nDeriv): 
    x, y=transpose(getPairs(points))
    derivs=json.loads(derivsString)
    coefs=derivs[nDeriv]

    x1=randX(min(x), max(x))
    y1=[f(coefs, x) for x in x1]
    
    return (
        dict(
            data=[
                dict(
                    x=x1,
                    y=y1,
                    name='Line of Best Fit'
                ),
            ],
            layout=dict(
                hovermode='closest'
            )
        ),
        f'f_{nDeriv} (x) = {getEq(coefs)}',
    )

In [88]:
@app.callback(
    Output('yDeriv', 'children'),
    [
        Input('xDeriv', 'value'),
        Input('derivs', 'children'),  
        Input('derivDrop', 'value'), 
    ]
)
def predictDeriv(x, derivsString, nDeriv):    
    derivs=json.loads(derivsString)
    coefs=derivs[nDeriv]

    return f'f_{nDeriv} ({x}) = {round(f(coefs, x), 9)}'

There you have it! You can see my web app at work by making a copy of this Google Colab file, or by visiting the page 'Least-Squares Data Modeling' in the 'Web Apps' section of my website.

Thank you for reading!

George Fane

In [89]:
app.run_server(mode='inline')